In [ ]:
import pandas as pd
import os, math, random
random.seed(2)
import numpy as np
import tensorflow as tf
import pyrsgis
from pyrsgis import raster
from tensorflow.compat.v1.keras.models import load_model
from tensorflow.keras.layers import Lambda, Reshape
from scipy import interpolate
import rasterio
from tensorflow.keras.layers import Layer
import keras.backend as K
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import PReLU, Concatenate, concatenate, LayerNormalization
from tensorflow.keras.layers import Conv1D, Dropout, Dense, BatchNormalization, ReLU
from tensorflow.keras.layers import Bidirectional, LSTM, GRU, Lambda, Reshape, Flatten, Activation
from tensorflow.keras.layers import Input, Add, TimeDistributed, MaxPooling1D, Softmax, Multiply
from tensorflow.compat.v1.keras.models import load_model
import sklearn

In [ ]:
# model = load_model('D:/Wiltang_2022/Paper_Data_Codes/Saved_Model/LC_Multivariate_Wonosobo_Encoder_ver01.h5', 
#                    custom_objects={'InstanceNormalization':InstanceNormalization})

# model = load_model('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Saved_Model/LC_DAS_Citarum_exp03_stable_reducedDate.h5')

# Load a new multispectral image
ds_cbi, img_cbi  = raster.read('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Wonosobo_Data/CBI_2014_2023_Wonosobo.tif')
ds_bsi, img_bsi  = raster.read('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Wonosobo_Data/BSI_2014_2023_Wonosobo.tif')
ds_evi2, img_evi2  = raster.read('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Wonosobo_Data/EVI2_2014_2023_Wonosobo.tif')
ds_mndwi, img_mndwi  = raster.read('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Wonosobo_Data/MNDWI_2014_2023_Wonosobo.tif')
ds_msavi, img_msavi  = raster.read('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Wonosobo_Data/MSAVI_2014_2023_Wonosobo.tif')
ds_nbr, img_nbr  = raster.read('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Wonosobo_Data/NBR_2014_2023_Wonosobo.tif')
ds_ndvi, img_ndvi  = raster.read('D:/DL_DASCitarum/Spasial_Vector/DAS_Citarum_Extract_Landsat/Wonosobo_Data/NDVI_2014_2023_Wonosobo.tif')

print(img_cbi.shape)
print(img_bsi.shape)
print(img_evi2.shape)
print(img_mndwi.shape)
print(img_msavi.shape)
print(img_nbr.shape)
print(img_ndvi.shape)

In [ ]:
img_cbi_reshape = img_cbi.reshape(img_cbi.shape[0],(img_cbi.shape[1]*img_cbi.shape[2])).T
img_bsi_reshape = img_bsi.reshape(img_bsi.shape[0],(img_bsi.shape[1]*img_bsi.shape[2])).T
img_evi2_reshape = img_evi2.reshape(img_evi2.shape[0],(img_evi2.shape[1]*img_evi2.shape[2])).T
img_mndwi_reshape = img_mndwi.reshape(img_mndwi.shape[0],(img_mndwi.shape[1]*img_mndwi.shape[2])).T
img_msavi_reshape = img_msavi.reshape(img_msavi.shape[0],(img_msavi.shape[1]*img_msavi.shape[2])).T
img_nbr_reshape = img_nbr.reshape(img_nbr.shape[0],(img_nbr.shape[1]*img_nbr.shape[2])).T
img_ndvi_reshape = img_ndvi.reshape(img_ndvi.shape[0],(img_ndvi.shape[1]*img_ndvi.shape[2])).T

In [ ]:
print(img_cbi_reshape.shape)
print(img_bsi_reshape.shape)
print(img_evi2_reshape.shape)
print(img_mndwi_reshape.shape)
print(img_msavi_reshape.shape)
print(img_nbr_reshape.shape)
print(img_ndvi_reshape.shape)

In [ ]:
print('Number of nan CBI before int:', len(~np.isnan(img_cbi_reshape)))
print('Number of nan BSI before int:', len(~np.isnan(img_bsi_reshape)))
print('Number of nan EVI2 before int:', len(~np.isnan(img_evi2_reshape)))
print('Number of nan MNDWI before int:', len(~np.isnan(img_mndwi_reshape)))
print('Number of nan MSAVI before int:', len(~np.isnan(img_msavi_reshape)))
print('Number of nan NBR before int:', len(~np.isnan(img_nbr_reshape)))
print('Number of nan NDVI before int:', len(~np.isnan(img_ndvi_reshape)))

In [ ]:
img_cbi_reshape_pd = pd.DataFrame(img_cbi_reshape)
img_bsi_reshape_pd = pd.DataFrame(img_bsi_reshape)
img_evi2_reshape_pd = pd.DataFrame(img_ndvi_reshape)
img_mndwi_reshape_pd = pd.DataFrame(img_mndwi_reshape)
img_msavi_reshape_pd = pd.DataFrame(img_msavi_reshape)
img_nbr_reshape_pd = pd.DataFrame(img_nbr_reshape)
img_ndvi_reshape_pd = pd.DataFrame(img_ndvi_reshape)

In [ ]:
%%time
#interpolate NaN values for each row

img_cbi_reshape_pd = pd.DataFrame(img_cbi_reshape)
img_bsi_reshape_pd = pd.DataFrame(img_bsi_reshape)
img_evi2_reshape_pd = pd.DataFrame(img_evi2_reshape)
img_mndwi_reshape_pd = pd.DataFrame(img_mndwi_reshape)
img_msavi_reshape_pd = pd.DataFrame(img_msavi_reshape)
img_nbr_reshape_pd = pd.DataFrame(img_nbr_reshape)
img_ndvi_reshape_pd = pd.DataFrame(img_ndvi_reshape)

img_cbi_reshape_pd.astype(float)
img_bsi_reshape_pd.astype(float)
img_evi2_reshape_pd.astype(float)
img_mndwi_reshape_pd.astype(float)
img_msavi_reshape_pd.astype(float) 
img_nbr_reshape_pd.astype(float)
img_ndvi_reshape_pd.astype(float)

img_cbi_int = img_cbi_reshape_pd.interpolate(method='spline', order=3, limit_direction='both', axis=1)
img_bsi_int = img_bsi_reshape_pd.interpolate(method='spline', order=3, limit_direction='both', axis=1)
img_evi2_int = img_evi2_reshape_pd.interpolate(method='spline', order=3, limit_direction='both', axis=1)
img_mndwi_int = img_mndwi_reshape_pd.interpolate(method='spline', order=3, limit_direction='both', axis=1)
img_msavi_int = img_msavi_reshape_pd.interpolate(method='spline', order=3, limit_direction='both', axis=1)
img_nbr_int = img_nbr_reshape_pd.interpolate(method='spline', order=3, limit_direction='both', axis=1)
img_ndvi_int = img_ndvi_reshape_pd.interpolate(method='spline', order=3, limit_direction='both', axis=1)


#check NaN Values
print('Number of nan CBI after int:', img_cbi_int.isnull().sum().sum())
print('Number of nan BSI after int:', img_bsi_int.isnull().sum().sum())
print('Number of nan EVI2 after int:', img_evi2_int.isnull().sum().sum())
print('Number of nan MNDWI after int:', img_mndwi_int.isnull().sum().sum())
print('Number of nan MSAVI after int:', img_msavi_int.isnull().sum().sum())
print('Number of nan NBR after int:', img_nbr_int.isnull().sum().sum())
print('Number of nan NDVI after int:', img_ndvi_int.isnull().sum().sum())

In [ ]:
img_cbi_int = np.asarray(img_cbi_int).astype(np.float64)
img_bsi_int = np.asarray(img_bsi_int).astype(np.float64)
img_evi2_int = np.asarray(img_evi2_int).astype(np.float64)
img_mndwi_int = np.asarray(img_mndwi_int).astype(np.float64)
img_msavi_int = np.asarray(img_msavi_int).astype(np.float64)
img_nbr_int = np.asarray(img_nbr_int).astype(np.float64)
img_ndvi_int = np.asarray(img_ndvi_int).astype(np.float64)

comb_all = np.stack((img_cbi_int, img_bsi_int, img_evi2_int,img_mndwi_int,img_msavi_int,
                    img_nbr_int,img_ndvi_int), axis=1)
comb_all_reshape = comb_all.reshape(comb_all.shape[0], comb_all.shape[2], comb_all.shape[1])

print(comb_all.shape)
print(comb_all_reshape.shape)

In [ ]:
%%time
pred_ = model.predict(comb_all_reshape)
agri_pred_ = pred_.argmax(axis=1)
img_pred = np.reshape(agri_pred_,(ds_ndvi.RasterYSize, ds_ndvi.RasterXSize))
print(img_pred.shape)
print(img_pred)
print(img_pred.min())
print(img_pred.max())

outFile = r'D:/Data_DBS/Citarum/Wonosobo_LCPred_ver03_stable_reducedDate.tif'
raster.export(img_pred, ds_ndvi, filename=outFile, dtype='int')